<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/CarObjectDetection/YoloV5CarOdjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**Clone YOLOv5**

In [ ]:
%cd /content/drive/MyDrive/TNT/CarObject

In [ ]:
%%time

!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies
!cp yolov5/requirements.txt ./

#**Library**


In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

#**Load Data**

In [ ]:
# 이미지 shape
img_h, img_w, num_channels = (380, 676, 3)

# 바운딩박스 데이터
bb_df = pd.read_csv('/content/drive/MyDrive/TNT/CarObject/train_solution_bounding_boxes (1).csv')

bb_df

#**Preprocessing**

In [ ]:
# 열 이름 : iamge -> image_id
bb_df.rename(columns = {'image' : 'image_id'}, inplace = True)
bb_df['image_id'] = bb_df['image_id'].apply(lambda x: x.split('.')[0]) # .jpg 떼고 id 저장

# 바운딩박스 중심점 (x_center, y_center)
# 바운딩박스 크기 (w, h)
bb_df['x_center'] = (bb_df['xmin']+bb_df['xmax'])/2
bb_df['y_center'] = (bb_df['ymin']+bb_df['ymax'])/2
bb_df['w'] = bb_df['xmax'] - bb_df['xmin']
bb_df['h'] = bb_df['ymax'] - bb_df['ymin']

# 클래스 열 추가하고 0으로 초기화
bb_df['classes'] = 0

# 상위 행 출력
bb_df.head()

In [ ]:
# 이미지 정규화
bb_df['x_center'] = bb_df['x_center']/img_w
bb_df['y_center'] = bb_df['y_center']/img_h

bb_df['w'] = bb_df['w']/img_w
bb_df['h'] = bb_df['h']/img_h

bb_df.head()

#**Split train&valid data**

In [ ]:
index = list(set(bb_df.image_id))

In [ ]:
len(index)

In [ ]:
train_idx = index[ : round(len(index*0.6))]
val_idx = index[round(len(index)*0.6):len(index)]

In [ ]:
print(len(train_idx),len(val_idx))

In [ ]:
base_path = '/content/drive/MyDrive/TNT/CarObject/'



for id, data in tqdm(bb_df.groupby('image_id')): # tqdm 상태바 표시
  if id in val_idx:
    save_folder = 'valid/'
  else:
    save_folder = 'train/'
  
  label_file_name = base_path + 'preprocessed_data/labels/' + save_folder + id + '.txt'

  with open(label_file_name, 'w+') as f:
    row = data[['classes', 'x_center', 'y_center', 'w', 'h']].astype(float).values
    row = row.astype(str)
    for i in range(len(row)):
      text = ' '.join(row[i])
      f.write(text)
      f.write('\n')
  
  image_path = base_path + 'preprocessed_data/images/' + save_folder + id + '.jpg'

  sh.copy(base_path + "training_images/{}.jpg".format(id), image_path)

In [ ]:
base_path = '/content/drive/MyDrive/TNT/CarObject/yolov5/preprocessed_data'



print("TRAIN LABELS : {}\nTRAIN IMAGES: {}\nVALID LABELS : {}\nVALID IMAGES: {}".format(
    len(os.listdir(base_path+'/labels/train')),
    len(os.listdir(base_path+'/images/train')),
    len(os.listdir(base_path+'/labels/valid')),
    len(os.listdir(base_path+'/images/valid')),
    ))

#**Training**

In [ ]:
%cd /content/drive/MyDrive/TNT/CarObject/yolov5

In [ ]:
%ls

In [ ]:
!python train.py --img 676 --batch 16 --epoch 20 --data /content/drive/MyDrive/TNT/CarObject/yolov5/data/coco.yaml --weights yolov5s.pt

#**Prediction**

In [ ]:
!python /content/drive/MyDrive/TNT/CarObject/yolov5/detect.py --weights yolov5/yolov5s.pt --img 676 --conf 0.4 --source /content/drive/MyDrive/TNT/CarObject/testing_images

In [ ]:
import random

def show_test_exp():
  rnd_images = random.choice(os.listdir('runs/detect/exp'))
  print(rnd_images)
  return Image('runs/detect/exp/'+rnd_images, width=500)

In [ ]:
show_test_exp()